# Spracovanie dát – hierarchická schéma CPV, rozdelenie do tréningovej a testovacej množiny

In [ ]:
import csv
import pandas as pd

# Funkcia na rekurzivne najdenie predkov v hierarchii schemy CPV
def najdi_predkov(kod, ddict, output_list):
    if kod in ddict:
        popis = ddict[kod]["popis"]
        kod_rodica = ddict[kod]["kod_rodica"]
        output_list.append({"kod": kod, "popis": popis})
        if kod_rodica:
            najdi_predkov(kod_rodica, ddict, output_list)

# Funkcia na nacitanie CSV suboru klasifikacnej schemy a vlozenie do slovnikovej struktury
def nacitaj_csv(subor):
    s_dict = {}
    with open(subor, newline="", encoding="utf-8") as csvfile:
        creader = csv.reader(csvfile)
        for row in creader:
            s_dict[row[0]] = {"kod_rodica": row[1], "popis": row[3]}
    return s_dict

# Nahraj CSV súbor a vytvor slovník
slovnik = nacitaj_csv("cpv_kategorie_fixed.csv")

# Ukazka najdenia kodu "42924300-2" v scheme CPV a rekurzivny vypis jeho predkov
kod_hladaneho = "42924300-2"
output_list = []
najdi_predkov(kod_hladaneho, slovnik, output_list)
df = pd.DataFrame(output_list).tail(3).iloc[::-1]
df.reset_index(drop=True, inplace=True)
print(df)
print("Dlzka DF je ...", len(df))

          kod                                              popis
0  42000000-6                                  Průmyslové stroje
1  42900000-5  Různá strojní zařízení pro všeobecné účely a s...
2  42920000-1  Přístroje k čištění lahví, balení a vážení a p...
Dlzka DF je ... 3


In [ ]:
cpv_kategorie = pd.read_csv("cpv_kategorie_fixed.csv")
cpv_kategorie["cpv_code_parent"].isnull().sum()

riadky_s_null = cpv_kategorie[cpv_kategorie.isnull().any(axis=1)]
len(riadky_s_null)

45

In [ ]:
# Prístup k hodnote v danom riadku (podľa indexu) a stĺpci (podľa názvu)
df.at[2, "kod"] + ": " + df.at[2, "popis"]

'42920000-1: Přístroje k čištění lahví, balení a vážení a postřikovací přístroje'

In [ ]:
# Nacitanie pripravenych dat na klasifikaciu textovych nazvov a popisov zakaziek
data = pd.read_csv("data-na-klasifikaciu2024.csv")
data

,title_with_description,cpv_full_code,cpv_full_description,type
0,Oprava bytových jader ve 2. NP v objektu B 30 ...,45453000-7,Opravy a modernizace budov,contract
1,Dům Chopin - rekonstrukce objektu - 2. etapa P...,45454100-5,Rekonstrukce budov,contract
2,UČEBNA FYZIKY; REGISTRAČNÍ ČÍSLO PROJEKTU: CZ....,39160000-1,Školní nábytek,contract
3,DATA_VPN PL_4357 Datové služby umožňující dat...,64200000-8,Telekomunikační služby,contract
4,Uzavření rámcových dohod na těžbu motorovou pi...,77211400-6,Kácení stromů,contract
...,...,...,...,...
59173,Předmětem plnění veřejné zakázky je na základ...,92521210-4,Ochrana exponátů,part
59174,Vícestranná rámcová dohoda na dodávky náhradní...,34731700-7,Součásti vrtulníků,part
59175,Předmětem veřejné zakázky je dodávka speciáln...,34114110-3,Záchranářská vozidla,part
59176,Předmětem zakázky je dodávka jednoho kusu Poj...,33100000-1,Zdravotnické přístroje,part


In [ ]:
# Odvodenie novych atributov DataFrame - kody CPV na prvej, druhej a tretej urovni
zoznam_urovni = list()

for index, value in data["cpv_full_code"].items():
    output_list = []
    najdi_predkov(value, slovnik, output_list)

    df = pd.DataFrame(output_list).tail(3).iloc[::-1]
    df.reset_index(drop=True, inplace=True)

    if len(df) >= 1:
        data.at[index, "cpv_uroven1"] = df.at[0, "kod"]
    if len(df) >= 2:
        data.at[index, "cpv_uroven2"] = df.at[1, "kod"]
    if len(df) >= 3:
        data.at[index, "cpv_uroven3"] = df.at[2, "kod"]

    zoznam_urovni.append(len(df))

In [ ]:
print(zoznam_urovni.count(3))
print(zoznam_urovni.count(2))
print(zoznam_urovni.count(1))

42445
8205
8528


In [ ]:
data

,title_with_description,cpv_full_code,cpv_full_description,type,cpv_uroven1,cpv_uroven2,cpv_uroven3
0,Oprava bytových jader ve 2. NP v objektu B 30 ...,45453000-7,Opravy a modernizace budov,contract,45000000-7,45400000-1,45450000-6
1,Dům Chopin - rekonstrukce objektu - 2. etapa P...,45454100-5,Rekonstrukce budov,contract,45000000-7,45400000-1,45450000-6
2,UČEBNA FYZIKY; REGISTRAČNÍ ČÍSLO PROJEKTU: CZ....,39160000-1,Školní nábytek,contract,39000000-2,39100000-3,39160000-1
3,DATA_VPN PL_4357 Datové služby umožňující dat...,64200000-8,Telekomunikační služby,contract,64000000-6,64200000-8,NaN
4,Uzavření rámcových dohod na těžbu motorovou pi...,77211400-6,Kácení stromů,contract,77000000-0,77200000-2,77210000-5
...,...,...,...,...,...,...,...
59173,Předmětem plnění veřejné zakázky je na základ...,92521210-4,Ochrana exponátů,part,92000000-1,92500000-6,92520000-2
59174,Vícestranná rámcová dohoda na dodávky náhradní...,34731700-7,Součásti vrtulníků,part,34000000-7,34700000-4,34730000-3
59175,Předmětem veřejné zakázky je dodávka speciáln...,34114110-3,Záchranářská vozidla,part,34000000-7,34100000-8,34110000-1
59176,Předmětem zakázky je dodávka jednoho kusu Poj...,33100000-1,Zdravotnické přístroje,part,33000000-0,33100000-1,NaN


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Zakodovanie kodov CPV v jednotlivych stlpcoch DataFrame na numericke hodnoty (label)
label_encoder = LabelEncoder()

def priprav_data_na_klasifikaciu(dataframe, level):
    column = ""
    if level == 1:
      column = "cpv_uroven1"
      dataframe = dataframe[["title_with_description", "cpv_full_code", "cpv_uroven1"]]
    elif level == 2:
      column = "cpv_uroven2"
      dataframe = dataframe[["title_with_description", "cpv_full_code", "cpv_uroven1", "cpv_uroven2"]]
    elif level == 3:
      column = "cpv_uroven3"
      dataframe = dataframe[["title_with_description", "cpv_full_code", "cpv_uroven1", "cpv_uroven2", "cpv_uroven3"]]
    dataframe = dataframe.dropna(how="any")
    column_label = column + "_label"

    value_counts = dataframe[column].value_counts()
    single_occurrences = value_counts[value_counts == 1].index.tolist()
    dataframe = dataframe[~dataframe[column].isin(single_occurrences)]

    if level > 1:
      dataframe["cpv_uroven1_label"] = label_encoder.fit_transform(dataframe["cpv_uroven1"])
    dataframe[column_label] = label_encoder.fit_transform(dataframe[column])
    return dataframe

data_cpv_uroven1 = priprav_data_na_klasifikaciu(data, 1)
data_cpv_uroven2 = priprav_data_na_klasifikaciu(data, 2)
data_cpv_uroven3 = priprav_data_na_klasifikaciu(data, 3)

In [ ]:
# Ulozenie spracovanych zaznamov verejnych zakaziek na druhej urovni klasifikacnej schemy do CSV suboru
data_cpv_uroven2.to_csv("data_cpv_uroven2.csv", encoding="utf-8", index=False)

In [ ]:
print(len(data_cpv_uroven1))
print(len(data_cpv_uroven2))
print(len(data_cpv_uroven3))

59178
50638
42357


In [ ]:
# Vypis vsetkych atributov a hodnot DataFrame so spracovanou druhou urovnou hierarchie schemy CPV
data_cpv_uroven2

,title_with_description,cpv_full_code,cpv_uroven1,cpv_uroven2,cpv_uroven1_label,cpv_uroven2_label
0,Oprava bytových jader ve 2. NP v objektu B 30 ...,45453000-7,45000000-7,45400000-1,22,126
1,Dům Chopin - rekonstrukce objektu - 2. etapa P...,45454100-5,45000000-7,45400000-1,22,126
2,UČEBNA FYZIKY; REGISTRAČNÍ ČÍSLO PROJEKTU: CZ....,39160000-1,39000000-2,39100000-3,17,93
3,DATA_VPN PL_4357 Datové služby umožňující dat...,64200000-8,64000000-6,64200000-8,29,163
4,Uzavření rámcových dohod na těžbu motorovou pi...,77211400-6,77000000-0,77200000-2,38,197
...,...,...,...,...,...,...
59173,Předmětem plnění veřejné zakázky je na základ...,92521210-4,92000000-1,92500000-6,43,225
59174,Vícestranná rámcová dohoda na dodávky náhradní...,34731700-7,34000000-7,34700000-4,13,72
59175,Předmětem veřejné zakázky je dodávka speciáln...,34114110-3,34000000-7,34100000-8,13,66
59176,Předmětem zakázky je dodávka jednoho kusu Poj...,33100000-1,33000000-0,33100000-1,12,62


In [ ]:
# Vypisu poctu vyskytov jednotlivych kodov skupin CPV na druhej urovni
data_cpv_uroven2["cpv_uroven2"].value_counts()

45200000-9    6225
33100000-1    6046
33600000-6    4185
34100000-8    2020
30200000-1    1804
              ... 
22600000-6       3
76100000-4       2
51900000-1       2
37800000-6       2
92200000-3       2
Name: cpv_uroven2, Length: 230, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

# Stratifikovane rozdelenie DataFrame do treningovej a testovacej mnoziny v pomere 70/30
X = data_cpv_uroven2.drop(columns=["cpv_uroven2_label"])
y = data_cpv_uroven2["cpv_uroven2_label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.70, stratify=y, random_state=24)

In [ ]:
# Vytvorenie novych DataFrame pre data z treningovej a testovacej mnoziny
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

In [ ]:
# Ukazka treningovej mnoziny dat verejnych obstaravani
train_df.head()

,title_with_description,cpv_full_code,cpv_uroven1,cpv_uroven2,cpv_uroven1_label,cpv_uroven2_label
45834,Forenzní software Předmětem plnění veřejné zak...,48900000-7,48000000-8,48900000-7,23,135
13635,REKONSTRUKCE UČEBEN PRO VÝUKU CIZÍCH JAZYKŮ A ...,39162000-5,39000000-2,39100000-3,17,93
11562,Multifunkční hala Výstavba nové multifukční sp...,45212225-9,45000000-7,45200000-9,22,124
42059,část 2-SO2 (obvod Správy tratí České Budějovic...,50225000-8,50000000-5,50200000-7,24,137
53046,Erlotinib Předmětem této části veřejné zakázky...,33652000-5,33000000-0,33600000-6,12,63


In [ ]:
train_df["cpv_uroven2_label"].nunique()

230

In [ ]:
# Ukazka treningovej mnoziny dat verejnych obstaravani
test_df.head()

,title_with_description,cpv_full_code,cpv_uroven1,cpv_uroven2,cpv_uroven1_label,cpv_uroven2_label
15446,KoPÚ v k. ú. Hostouň u Horšovského Týna a Tasn...,71250000-5,71000000-8,71200000-0,33,172
42403,Dodávky izolátorů VVN keramických podpěrných p...,44111511-6,44000000-0,44100000-1,21,115
16069,FNUSA - Nábytkové vybavení - objekt P Předměte...,39130000-2,39000000-2,39100000-3,17,93
50364,Část 3 - Monitory Monitory,30231310-3,30000000-9,30200000-1,9,50
32290,"Oprava silnice II/606, Pomezí nad Ohří - SO 10...",45233140-2,45000000-7,45200000-9,22,124


In [ ]:
test_df["cpv_uroven2_label"].nunique()

230

In [ ]:
# Ulozenie treningovej a testovacej mnoziny dat verejnych obstaravani do CSV suboru
train_df.to_csv("train_df.csv", index=False)
test_df.to_csv("test_df.csv", index=False)

In [ ]:
# Vytvorenie dvojic hodnot slovnika z DataFrame - atributy "label" a "encoded_label"
unique_pairs_dict = {}

for label, encoded_label in zip(data_cpv_uroven1["cpv_uroven1"], data_cpv_uroven1["cpv_uroven1_label"]):
    unique_pairs_dict[encoded_label] = label

print("Slovník unikátnych dvojíc hodnôt:")
print(unique_pairs_dict)

Slovník unikátnych dvojíc hodnôt:
{22: '45000000-7', 17: '39000000-2', 29: '64000000-6', 38: '77000000-0', 10: '31000000-6', 9: '30000000-9', 23: '48000000-8', 13: '34000000-7', 34: '72000000-5', 42: '90000000-7', 40: '80000000-4', 1: '09000000-3', 31: '66000000-0', 16: '38000000-5', 12: '33000000-0', 39: '79000000-4', 19: '42000000-6', 11: '32000000-3', 24: '50000000-5', 5: '18000000-9', 33: '71000000-8', 14: '35000000-4', 21: '44000000-0', 44: '98000000-3', 20: '43000000-3', 27: '60000000-8', 43: '92000000-1', 26: '55000000-0', 8: '24000000-4', 2: '14000000-1', 15: '37000000-8', 6: '19000000-6', 4: '16000000-5', 7: '22000000-0', 28: '63000000-9', 3: '15000000-8', 41: '85000000-9', 35: '73000000-2', 37: '76000000-3', 32: '70000000-1', 25: '51000000-9', 0: '03000000-1', 36: '75000000-6', 18: '41000000-9', 30: '65000000-3'}


In [ ]:
import graphviz
from graphviz import Source

# Funkcia na graficku vizualizaciu potomkov zadaneho kodu klasifikacnej schemy CPV
def vytvor_strom(ddict, kod, graf=None):
    if graf is None:
        graf = graphviz.Digraph(format="png")

    if kod in ddict:
        k, popis = kod, ddict[kod]["popis"]
        graf.node(k, label=f"{k}\n{popis}")
        for k, v in ddict.items():
            if v["kod_rodica"] == kod:
                potomok_kod = k
                graf.edge(kod, potomok_kod)
                vytvor_strom(ddict, potomok_kod, graf)
    return graf

# Vizualizacia stromovej hierarchie CPV schemy pre pozadovany kod
slovnik = nacitaj_csv("cpv_kategorie_fixed.csv")
kod_hladaneho = "03320000-8"
strom = vytvor_strom(slovnik, kod_hladaneho)

# Ulozenie grafickeho znazornenia stromu do PNG suboru
strom.render("stromCPV", format="png", cleanup=True)